In [1]:
import pandas as pd                                                                                                                                                              
import numpy as np                                                                                                                                                               
import wfdb                                                                                                                                                                      
import ast
from tqdm import tqdm
tqdm.pandas()                                                                                                                                                              

In [2]:
def load_raw_data(df, sampling_rate, path):                                                                                                                                      
    if sampling_rate == 100:                                                                                                                                                     
        data = [wfdb.rdsamp(path+f) for f in tqdm(df.filename_lr)]                                                                                                                     
    else:                                                                                                                                                                        
        data = [wfdb.rdsamp(path+f) for f in tqdm(df.filename_hr)]                                                                                                                     
    data = np.array([signal for signal, meta in data])                                                                                                                           
    return data                                                                                                                                                                  
                                                                                                                                                                                 
path = '/home/moritz/data/physionet.org/files/ptb-xl/1.0.3/'
sampling_rate = 100            

# load and convert annotation data                                                                                                                                               
df = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')                                                                                                                   
df.scp_codes = df.scp_codes.apply(lambda x: ast.literal_eval(x))
                                                                                                                                                                               
# Load raw signal data                                                                                                                                                           
X = load_raw_data(df, sampling_rate, path)                                                                                                                                      

100%|██████████| 21799/21799 [00:32<00:00, 664.85it/s]


In [3]:
np.mean(X), np.std(X), np.min(X), np.max(X)

(np.float64(-0.0007808800556600472),
 np.float64(0.23539237334110466),
 np.float64(-20.032),
 np.float64(17.212))

In [4]:
# prepare binary columns for the five superclasses
df["NORM"] = 0
df["MI"] = 0
df["STTC"] = 0
df["CD"] = 0
df["HYP"] = 0

In [5]:
# Load scp_statements.csv for diagnostic aggregation                                                                                                                             
agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)                                                                                                                     
agg_df = agg_df[agg_df.diagnostic == 1]      

In [6]:
# aggregate diagnoses into superclasses with a lookup into agg_df
def aggregate_diagnostic(row):
    for key in row.scp_codes.keys():                                                                                                                                                     
        if key in agg_df.index:                                                                                                                                                  
            respective_superclass = str(agg_df.loc[key].diagnostic_class)
            if respective_superclass[0] == '[':
                respective_superclass = ast.literal_eval(respective_superclass)[0]
            row[respective_superclass] = 1
    return row

In [7]:
# Apply diagnostic superclass                                                                                                                                                    
df = df.progress_apply(aggregate_diagnostic, axis=1)

100%|██████████| 21799/21799 [00:02<00:00, 10232.50it/s]


In [8]:
df.CD.value_counts(dropna=False), df.HYP.value_counts(dropna=False), df.MI.value_counts(dropna=False), df.NORM.value_counts(dropna=False), df.STTC.value_counts(dropna=False)

(CD
 0    16901
 1     4898
 Name: count, dtype: int64,
 HYP
 0    19150
 1     2649
 Name: count, dtype: int64,
 MI
 0    16330
 1     5469
 Name: count, dtype: int64,
 NORM
 0    12285
 1     9514
 Name: count, dtype: int64,
 STTC
 0    16564
 1     5235
 Name: count, dtype: int64)

In [9]:
# official train-test split
test_fold = 10
# inputs
X_train = X[np.where(df.strat_fold != test_fold)]
X_test = X[np.where(df.strat_fold == test_fold)]
# targets
train_df = df[(df.strat_fold != test_fold)]
test_df = df[(df.strat_fold == test_fold)]

In [10]:
train_df = train_df.reset_index()
test_df = test_df.reset_index()
assert len(X_train) == len(train_df)
assert len(X_test) == len(test_df)
len(train_df), len(test_df)

(19601, 2198)

In [11]:
train_df.to_csv("/home/moritz/repositories/med_leak/data/csv/ptb-xl_train.csv")
test_df.to_csv("/home/moritz/repositories/med_leak/data/csv/ptb-xl_test.csv")

In [12]:
np.save(f"{path}X_{sampling_rate}_train.npy", X_train)
np.save(f"{path}X_{sampling_rate}_test.npy", X_test)